#SoMe Topic Modeling Notebook | Release canvas 1 📖

## Installations and Libraries 💽

In [3]:
import time 
# Time the running of everything
start_of_notebook_time = time.time()

In [4]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)


C:\Users\user1\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.

C:\Users\user1\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

C:\Users\user1\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



##Data Cleaning 🧹

In [5]:
df = pd.read_csv('jan_uber_positive.csv')
df = pd.DataFrame({"original_tweets":df['Tweets']})
df.head()

#Removing emojies from text
#Refrence 1 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 2 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
df['emoji_free_tweets'] = df['original_tweets'].apply(call_emoji_free)

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)

df

,original_tweets,emoji_free_tweets,url_free_tweets
0,_Support\n_NYC\n_India\n \n\nMy account has go...,_Support _NYC _India My account has gone too l...,_Support _NYC _India My account has gone too l...
1,"Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ..."
2,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...
3,"_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c..."
4,"Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ..."
...,...,...,...
6467,"Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio..."
6468,is anyone alive?,is anyone alive?,is anyone alive?
6469,"Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe..."
6470,"Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap..."


##Tokenizing 🕵🏻‍♂

In [6]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')


In [7]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['url_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens

# Timing End
program_end_time = time.time()

# View df
df

,original_tweets,emoji_free_tweets,url_free_tweets,tokens
0,_Support\n_NYC\n_India\n \n\nMy account has go...,_Support _NYC _India My account has gone too l...,_Support _NYC _India My account has gone too l...,"[_support, _nyc, _india, account, gone, long, ..."
1,"Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","[let, corporate, personal, life,, need, help, ..."
2,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,"[india, hacked, uber, account, u]"
3,"_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","[_luv, hey, rahul,, kindly, share, registered,..."
4,"Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","[hey, biju,, safety, priority,, recommended, 2..."
...,...,...,...,...
6467,"Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","[hey,, send, note, help, section, app, direct,..."
6468,is anyone alive?,is anyone alive?,is anyone alive?,[alive?]
6469,"Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","[hey, rajiv,, share, registered, details, dire..."
6470,"Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","[hey,, sorry, hear, weren't, able, apply, prom..."


In [16]:
import re
df['url_free_tweets']=df['url_free_tweets'].apply(lambda x: re.sub(r"_", ' ', str(x)))
df

,original_tweets,emoji_free_tweets,url_free_tweets,tokens
0,_Support\n_NYC\n_India\n \n\nMy account has go...,_Support _NYC _India My account has gone too l...,Support NYC India My account has gone too l...,"['support', 'nyc', 'india', 'account', 'gone',..."
1,"Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","['let', 'corporate', 'personal', 'life,', 'nee..."
2,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,"['india', 'hacked', 'uber', 'account', 'u']"
3,"_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","luv Hey Rahul, kindly share your registered c...","['luv', 'hey', 'rahul,', 'kindly', 'share', 'r..."
4,"Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","['hey', 'biju,', 'safety', 'priority,', 'recom..."
...,...,...,...,...
6467,"Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","['hey,', 'send', 'note', 'help', 'section', 'a..."
6468,is anyone alive?,is anyone alive?,is anyone alive?,['alive?']
6469,"Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","['hey', 'rajiv,', 'share', 'registered', 'deta..."
6470,"Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","['hey,', 'sorry', 'hear', ""weren't"", 'able', '..."


In [6]:
# See how long it took
print(program_end_time - program_start_time, "seconds to finish")

3.8468899726867676 seconds to finish


##Lemmatization🇬🇧

In [7]:
# Refrence 4 : https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column

# Timing Start
program_start_time = time.time()

# Make tokens a string again
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
# df[['original_tweet', 'lemmas_back_to_text']]

# Timing End
program_end_time = time.time()


In [8]:
#Printing Lemmetization Time
print(program_end_time - program_start_time, "seconds to finish")

197.85924100875854 seconds to finish


In [9]:
# Timing Start
program_start_time = time.time()

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(r'[^a-zA-Z 0-9]', '', text)
    tokens = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub(r'\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub(r'@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

# Timing End
program_end_time = time.time()

# View those tokens (the 4th column)
df

,original_tweets,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,_Support\n_NYC\n_India\n \n\nMy account has go...,_Support _NYC _India My account has gone too l...,_Support _NYC _India My account has gone too l...,"[_support, _nyc, _india, account, gone, long, ...",_support _nyc _india account gone long fixed k...,"[support, nyc, india, account, go, long, fix, ...",support nyc india account go long fix know,"[support, nyc, india, account, go, long, fix, ..."
1,"Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","Let it corporate or personal life, in need of ...","[let, corporate, personal, life,, need, help, ...","let corporate personal life, need help crisis,...","[let, corporate, personal, life, need, help, c...",let corporate personal life need help crisis h...,"[let, corporate, personal, life, need, help, c..."
2,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,Someone in India really hacked my uber account...,"[india, hacked, uber, account, u]",india hacked uber account u,"[india, hack, uber, account, u]",india hack uber account u,"[india, hack, uber, account, u]"
3,"_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","_luv Hey Rahul, kindly share your registered c...","[_luv, hey, rahul,, kindly, share, registered,...","_luv hey rahul, kindly share registered contac...","[luv, hey, rahul, kindly, share, register, con...",luv hey rahul kindly share register contact de...,"[luv, hey, rahul, kindly, share, register, con..."
4,"Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","Hey Biju, with safety as our top priority, it ...","[hey, biju,, safety, priority,, recommended, 2...","hey biju, safety priority, recommended 2 peopl...","[hey, biju, safety, priority, recommend, 2, pe...",hey biju safety priority recommend 2 people ub...,"[hey, biju, safety, priority, recommend, 2, pe..."
...,...,...,...,...,...,...,...,...
6467,"Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","Hey, please send us a note via the help sectio...","[hey,, send, note, help, section, app, direct,...","hey, send note help section app direct message...","[hey, send, note, help, section, app, direct, ...",hey send note help section app direct message ...,"[hey, send, note, help, section, app, direct, ..."
6468,is anyone alive?,is anyone alive?,is anyone alive?,[alive?],alive?,[alive],alive,[alive]
6469,"Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","Hey Rajiv, could you please share your registe...","[hey, rajiv,, share, registered, details, dire...","hey rajiv, share registered details direct mes...","[hey, rajiv, share, register, detail, direct, ...",hey rajiv share register detail direct message...,"[hey, rajiv, share, register, detail, direct, ..."
6470,"Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","Hey, sorry to hear that you weren't able to ap...","[hey,, sorry, hear, weren't, able, apply, prom...","hey, sorry hear weren't able apply promotional...","[hey, sorry, hear, able, apply, promotional, c...",hey sorry hear able apply promotional code sha...,"[hey, sorry, hear, able, apply, promotional, c..."


In [10]:
#Printing Tokenization Time
print(program_end_time - program_start_time, "seconds to finish")


1.7772495746612549 seconds to finish


##Topic Modeling ㊙️

###id2word 📒

In [11]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))

9049


In [12]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

3696


###Corpus Object & Generating Base Model Topics 📚

In [13]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

####Base Model

In [14]:
# Timing Start
base_model_program_start_time = time.time()

# Instantiating a LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

# Timing End
base_model_program_end_time = time.time()


In [15]:
#Printing First Model Time
base_model_runtime = round(base_model_program_end_time - base_model_program_start_time, 2)
print(base_model_runtime)


47.92


In [16]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]


In [17]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]


In [18]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
hey help message direct kindly share detail app register section

------ Topic 1 ------
new health make simple manage premium device seamless style express

------ Topic 2 ------
service support hey uber india driver essential zone help kindly

------ Topic 3 ------
india uber driver ride good cab support work delhi great

------ Topic 4 ------
uber service guideline ride hey available city notice driver thank



In [19]:
# Compute Perplexity
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)



Perplexity:  -6.286402913419075

Coherence Score:  0.4561066501444625


#### Base Model Topic Distance Visualization 📈

In [20]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.254829 -0.050532       1        1  34.636896
3     -0.160690  0.018001       2        1  24.204671
4      0.019593  0.151675       3        1  14.886369
2     -0.026074  0.038221       4        1  14.094508
1     -0.087659 -0.157365       5        1  12.177556, topic_info=        Term         Freq        Total Category  logprob  loglift
3      india  1711.000000  1711.000000  Default  30.0000  30.0000
108  message  1121.000000  1121.000000  Default  29.0000  29.0000
106   direct  1047.000000  1047.000000  Default  28.0000  28.0000
25      uber  2283.000000  2283.000000  Default  27.0000  27.0000
206  service   954.000000   954.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
113      app    66.917623  1034.675625   Topic5  -5.0331  -0.6328
14      help    75.122076  1634.611381   Topic5  -4.9175  -0.9745
314     able    33.113570   187.763406   Topic5  -5.7366   0.3703
180     ride    48.430635   652.552164   Topic5  -5.3564  -0.4952
29       hey    50.133662  2062.439204   Topic5  -5.3219  -1.6114

[340 rows x 6 columns], token_table=      Topic      Freq Term
term                      
37        1  0.073789    2
37        2  0.209069    2
37        3  0.491927    2
37        4  0.049193    2
37        5  0.172174    2
...     ...       ...  ...
58        3  0.025506    |
58        5  0.025506    |
618       3  0.922203    £
2794      3  0.859110   है
2794      4  0.095457   है

[935 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 3, 2])

###Grid Seach 🔍

In [21]:
lemmas_df = df['lemmas_back_to_text']
print(type(lemmas_df[0]))


<class 'str'>


In [22]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df['lemmas_back_to_text'])


In [23]:
gs_start_time = time.time()

# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             iid=True, n_jobs=1,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)

gs_end_time = time.time()


C:\Users\user1\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.



In [24]:
print(gs_end_time - gs_start_time, "seconds to finish")


1556.9325757026672 seconds to finish


In [55]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
# Best Model's Params:  {'learning_decay': 0.9, 'n_topics': 10}

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -249400.0726756264
Model Perplexity:  752.7927832092748


###Hyperparameter Tuning 🦾

####Model iteration 1 (number of topics)


#####1.0 Topics = 5

In [56]:
# Let's start with parameter tuning for the LDA model and,
# find an optimal number of topics to reach the best coherence score

# Define chunksize and passes
# Chunksize is Number of documents to be used in each training chunk
# Passes is Number of passes through the corpus during training

# Timing Start
model_1_0_start_time = time.time()

model_1_0 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=5,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_0_end_time = time.time()


In [57]:
#Printing First Model Time
model_1_0_runtime = round(model_1_0_end_time - model_1_0_start_time, 2)
print(model_1_0_runtime)


42.55


In [58]:
# Filtering for words 
words_1_0 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_0.print_topics()]

# Create Topics
topics_1_0 = [' '.join(t[0:10]) for t in words_1_0]

# Getting the topics
for id, t in enumerate(topics_1_0): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
uber service read hey guideline safety available ride driver india

------ Topic 1 ------
india uber driver work good ola support time car come

------ Topic 2 ------
hey message direct share register kindly detail follow sorry number

------ Topic 3 ------
help app hey section send kindly response way concern uber

------ Topic 4 ------
health new make premium simple manage device seamless style express



In [59]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_0_perplexity = model_1_0.log_perplexity(corpus)
print('\nPerplexity: ', model_1_0_perplexity) 

# Compute Coherence Score
coherence_model_1_0 = CoherenceModel(model=model_1_0, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_0 = coherence_model_1_0.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_0)



Perplexity:  -6.302528402577665

Coherence Score:  0.5275585254774091


In [60]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_0, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.197109  0.130403       1        1  24.083682
3      0.179441 -0.003647       2        1  23.309216
0     -0.129689  0.112936       3        1  19.919597
2      0.297864  0.010994       4        1  19.002178
4     -0.150506 -0.250687       5        1  13.685327, topic_info=         Term         Freq        Total Category  logprob  loglift
108   message  1083.000000  1083.000000  Default  30.0000  30.0000
106    direct  1018.000000  1018.000000  Default  29.0000  29.0000
3       india  1813.000000  1813.000000  Default  28.0000  28.0000
36      share   823.000000   823.000000  Default  27.0000  27.0000
35   register   724.000000   724.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
45     safety   107.498489   412.355747   Topic5  -4.6758   0.6444
25       uber   132.651029  2243.400712   Topic5  -4.4656  -0.8392
29        hey    98.277832  2022.717660   Topic5  -4.7655  -1.0356
101      good    79.540714   370.626715   Topic5  -4.9770   0.4499
40     follow    82.920744   620.130414   Topic5  -4.9354  -0.0232

[314 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
656       1  0.978528  .twitter.com
37        1  0.256356             2
37        3  0.179449             2
37        4  0.006409             2
37        5  0.557575             2
...     ...       ...           ...
69        1  0.844569          year
69        3  0.144783          year
978       1  0.989846        zomato
1854      3  0.991419          zone
2794      5  0.956307            है

[528 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

#####1.1 Topics = 10

In [61]:
#Increasing number of topics to 10
#Timing Start
model_1_1_start_time = time.time()

model_1_1 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=10,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_1_end_time = time.time()


In [62]:
#Printing First Model Time
model_1_1_runtime = round(model_1_1_end_time - model_1_1_start_time, 2)
print(model_1_1_runtime)


35.24


In [63]:
# Filtering for words 
words_1_1 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_1.print_topics()]

# Create Topics
topics_1_1 = [' '.join(t[0:10]) for t in words_1_1]

# Getting the topics
for id, t in enumerate(topics_1_1): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
uber service read hey guideline available safety ride city accordance

------ Topic 1 ------
india uber good great work ola sir driver company job

------ Topic 2 ------
message detail direct share hey register kindly follow trip sorry

------ Topic 3 ------
help app section hey send kindly response way concern update

------ Topic 4 ------
thank issue uber support team day ac need driver thanks

------ Topic 5 ------
new health make simple manage seamless device premium express style

------ Topic 6 ------
hey number message direct kindly register follow concern share look

------ Topic 7 ------
driver ride cab cancel india uber book pay trip location

------ Topic 8 ------
thank driver uber india hai action soon work bring ko

------ Topic 9 ------
driver uber india safety customer rider time ride safe well



In [64]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_1_perplexity = model_1_1.log_perplexity(corpus)
print('\nPerplexity: ', model_1_1_perplexity) 

# Compute Coherence Score
coherence_model_1_1 = CoherenceModel(model=model_1_1, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_1 = coherence_model_1_1.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_1)



Perplexity:  -6.342547666613483

Coherence Score:  0.4971956661440924


In [65]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_1, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.196690  0.061013       1        1  15.755607
0      0.065806 -0.013658       2        1  13.307734
1      0.199067  0.051752       3        1  12.934771
2     -0.288591  0.039653       4        1  12.083315
6     -0.286933 -0.006331       5        1   9.882181
7      0.138294  0.084784       6        1   9.209513
5      0.052587 -0.395667       7        1   8.778215
4      0.105154  0.073674       8        1   7.747472
9      0.083064 -0.002689       9        1   6.310030
8      0.128241  0.107469      10        1   3.991161, topic_info=           Term         Freq        Total Category  logprob  loglift
106      direct  1040.000000  1040.000000  Default  30.0000  30.0000
108     message  1110.000000  1110.000000  Default  29.0000  29.0000
3         india  1601.000000  1601.000000  Default  28.0000  28.0000
29          hey  2060.000000  2060.000000  Default  27.0000  27.0000
28       detail   954.000000   954.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
244      credit    20.040178    55.138064  Topic10  -5.1233   2.2090
774      accept    18.011152    35.830484  Topic10  -5.2301   2.5333
283       happy    23.458662   199.983835  Topic10  -4.9658   1.0781
258  understand    23.082148   183.779760  Topic10  -4.9820   1.1464
14         help    30.225598  1362.508543  Topic10  -4.7124  -0.5873

[641 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
656       3  0.980666  .twitter.com
2731      6  0.959522             0
760       1  0.145719            10
760       4  0.145719            10
760       6  0.686959            10
...     ...       ...           ...
3669     10  0.967616            ने
1177      8  0.895400            ये
3670     10  0.967614          लेकर
3671     10  0.967614      वनइंडिया
2794      8  0.976194            है

[1254 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 7, 8, 6, 5, 10, 9])

#####1.2 Topics = 15

In [66]:
#Increasing number of topics to 15
#Timing Start
model_1_2_start_time = time.time()

model_1_2 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=15,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_2_end_time = time.time()


In [67]:
#Printing First Model Time
model_1_2_runtime = round(model_1_2_end_time - model_1_2_start_time, 2)
print(model_1_2_runtime)


43.61


In [68]:
# Filtering for words 
words_1_2 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_2.print_topics()]

# Create Topics
topics_1_2 = [' '.join(t[0:10]) for t in words_1_2]

# Getting the topics
for id, t in enumerate(topics_1_2): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
uber service available guideline india read ride mean city government

------ Topic 1 ------
india uber work driver love pls year car delivery drive

------ Topic 2 ------
share message direct hey detail register kindly sorry follow hear

------ Topic 3 ------
help uber driver happy account drop hey hope india trip

------ Topic 4 ------
cancel charge ride driver uber trip day pay ask india

------ Topic 5 ------
kindly hey uber trip allow update fare cab support app

------ Topic 6 ------
number hey message direct register early follow uber mobile send

------ Topic 7 ------
account uber issue trip provide hey book assist request face

------ Topic 8 ------
thank driver action india people give uber work understand take

------ Topic 9 ------
driver good great safety job uber ride time take rider

------ Topic 10 ------
hey help app section concern kindly send way response update

------ Topic 11 ------
new make health simple manage seamless device premium expres

In [69]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_2_perplexity = model_1_2.log_perplexity(corpus)
print('\nPerplexity: ', model_1_2_perplexity) 

# Compute Coherence Score
coherence_model_1_2 = CoherenceModel(model=model_1_2, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_2 = coherence_model_1_2.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_2)



Perplexity:  -6.333067227401998

Coherence Score:  0.47320311850378516


In [70]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_2, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.257767  0.021939       1        1  11.081918
2      0.258005 -0.018728       2        1  10.928303
14    -0.166646  0.043306       3        1  10.880934
1     -0.191255  0.045495       4        1   7.968948
6      0.223135 -0.022323       5        1   7.588679
4     -0.137496  0.075067       6        1   7.550233
0     -0.126235  0.024148       7        1   7.504817
3     -0.004009  0.072329       8        1   6.222043
5      0.070122  0.078962       9        1   5.702612
11    -0.096067 -0.471089      10        1   5.697302
13    -0.047012  0.058966      11        1   4.665906
9     -0.089988  0.033184      12        1   4.362549
8     -0.171945  0.041839      13        1   3.525113
7      0.045612  0.054299      14        1   3.266773
12     0.176013 -0.037395      15        1   3.053870, topic_info=             Term         Freq        Total Category  logprob  loglift
106        direct  1057.000000  1057.000000  Default  30.0000  30.0000
108       message  1133.000000  1133.000000  Default  29.0000  29.0000
35       register   765.000000   765.000000  Default  28.0000  28.0000
36          share   873.000000   873.000000  Default  27.0000  27.0000
175        driver  1040.000000  1040.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
180          ride    29.740713   624.707731  Topic15  -4.4609   0.4440
218  cancellation    20.557513   129.868983  Topic15  -4.8302   1.6455
29            hey    29.079711  2116.874827  Topic15  -4.4833  -0.7989
198  availability    19.460133   142.066122  Topic15  -4.8850   1.5008
25           uber    21.788623  2142.279087  Topic15  -4.7720  -1.0995

[951 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2731     10  0.918571         0
760       6  0.657994        10
760       8  0.308435        10
834       3  0.050224       100
834       6  0.904033       100
...     ...       ...       ...
3670     15  0.891920      लेकर
3671     15  0.891919  वनइंडिया
2828      6  0.903507        से
3672     15  0.891915     हिंदी
2794      6  0.949548        है

[1851 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 3, 15, 2, 7, 5, 1, 4, 6, 12, 14, 10, 9, 8, 13])

##### 1.3 Topics = 20

In [71]:
#Increasing number of topics to 20
#Timing Start
model_1_3_start_time = time.time()

model_1_3 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=20,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_3_end_time = time.time()


In [72]:
#Printing First Model Time
model_1_3_runtime = round(model_1_3_end_time - model_1_3_start_time, 2)
print(model_1_3_runtime)


48.78


In [73]:
# Filtering for words 
words_1_3 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_3.print_topics()]

# Create Topics
topics_1_3 = [' '.join(t[0:10]) for t in words_1_3]

# Getting the topics
for id, t in enumerate(topics_1_3): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
service uber read available guideline ride city mean government notice

------ Topic 1 ------
india uber delivery pls local love encourage eat zomato food

------ Topic 2 ------
trip time platform india date kind detail experience share uber

------ Topic 3 ------
help app section hey trip kindly update fare review happy

------ Topic 4 ------
driver ride charge pay ac day reply issue ask cancel

------ Topic 5 ------
hey kindly travel cab support mumbai essential allow ride availability

------ Topic 6 ------
uber safety 2 hey people priority vehicle distance follow local

------ Topic 7 ------
issue account know hey india cancel uber app let kindly

------ Topic 8 ------
thank driver india uber think action trip happen help respect

------ Topic 9 ------
message direct send kind experience definitely registered detail hey address

------ Topic 10 ------
help hey kindly app check way send section concern response

------ Topic 11 ------
new health make simple man

In [74]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_3_perplexity = model_1_3.log_perplexity(corpus)
print('\nPerplexity: ', model_1_3_perplexity) 

# Compute Coherence Score
coherence_model_1_3 = CoherenceModel(model=model_1_3, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_3 = coherence_model_1_3.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_3)



Perplexity:  -6.322442138314105

Coherence Score:  0.4895762869976711


In [75]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_3, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.301499 -0.013672       1        1  15.743518
0     -0.117465  0.051235       2        1   7.587195
14    -0.156432  0.045076       3        1   7.372338
5     -0.008016  0.066562       4        1   6.599262
11    -0.112941 -0.476169       5        1   5.543840
3      0.199853  0.016801       6        1   5.342727
1     -0.180798 -0.004534       7        1   4.952686
10     0.187303  0.018472       8        1   4.742786
4     -0.126678  0.036757       9        1   4.710837
17    -0.182014  0.060560      10        1   4.383433
19    -0.142768  0.034225      11        1   4.344817
7     -0.019331  0.052022      12        1   3.903939
13     0.020026  0.035213      13        1   3.875106
15    -0.063903  0.033820      14        1   3.704522
6     -0.095621  0.040341      15        1   3.294883
12     0.271118 -0.045269      16        1   2.997403
2      0.038838  0.011192      17        1   2.960202
9      0.138506 -0.002113      18        1   2.928769
8     -0.103464  0.041790      19        1   2.663030
16     0.152288 -0.002309      20        1   2.348706, topic_info=         Term         Freq        Total Category  logprob  loglift
108   message  1235.000000  1235.000000  Default  30.0000  30.0000
106    direct  1161.000000  1161.000000  Default  29.0000  29.0000
35   register   884.000000   884.000000  Default  28.0000  28.0000
36      share  1003.000000  1003.000000  Default  27.0000  27.0000
14       help  1367.000000  1367.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
278      trip    13.350649   712.326715  Topic20  -4.9993  -0.2257
3       india    13.350121  1536.369423  Topic20  -4.9993  -0.9943
314      able    10.921954   239.939284  Topic20  -5.2001   0.6617
276      date    10.506997   143.686130  Topic20  -5.2388   1.1357
101      good    10.309326   304.825999  Topic20  -5.2578   0.3646

[1240 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
969      17  0.900055        .i
2731      1  0.862450         0
70        3  0.216649         1
70        4  0.034208         1
70        6  0.068415         1
...     ...       ...       ...
3670     16  0.896597      लेकर
3671     16  0.896605  वनइंडिया
2828      9  0.945537        से
3672     16  0.896616     हिंदी
2794      9  0.942204        है

[2295 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 1, 15, 6, 12, 4, 2, 11, 5, 18, 20, 8, 14, 16, 7, 13, 3, 10, 9, 17])

##### 1.4 Topics = 25

In [76]:
#Increasing number of topics to 25
#Timing Start
model_1_4_start_time = time.time()

model_1_4 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=25,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_4_end_time = time.time()


In [77]:
#Printing First Model Time
model_1_4_runtime = round(model_1_4_end_time - model_1_4_start_time, 2)
print(model_1_4_runtime)


41.62


In [78]:
# Filtering for words 
words_1_4 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_4.print_topics()]

# Create Topics
topics_1_4 = [' '.join(t[0:10]) for t in words_1_3]

# Getting the topics
for id, t in enumerate(topics_1_4): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
service uber read available guideline ride city mean government notice

------ Topic 1 ------
india uber delivery pls local love encourage eat zomato food

------ Topic 2 ------
trip time platform india date kind detail experience share uber

------ Topic 3 ------
help app section hey trip kindly update fare review happy

------ Topic 4 ------
driver ride charge pay ac day reply issue ask cancel

------ Topic 5 ------
hey kindly travel cab support mumbai essential allow ride availability

------ Topic 6 ------
uber safety 2 hey people priority vehicle distance follow local

------ Topic 7 ------
issue account know hey india cancel uber app let kindly

------ Topic 8 ------
thank driver india uber think action trip happen help respect

------ Topic 9 ------
message direct send kind experience definitely registered detail hey address

------ Topic 10 ------
help hey kindly app check way send section concern response

------ Topic 11 ------
new health make simple man

In [79]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_4_perplexity = model_1_4.log_perplexity(corpus)
print('\nPerplexity: ', model_1_4_perplexity) 

# Compute Coherence Score
coherence_model_1_4 = CoherenceModel(model=model_1_4, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_4 = coherence_model_1_4.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_4)



Perplexity:  -6.311173451379648

Coherence Score:  0.4518313625062108


In [80]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_4, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.331506  0.039781       1        1  13.907166
12    -0.296598  0.015999       2        1   6.298102
0      0.098761  0.088073       3        1   6.177779
14     0.182803  0.039186       4        1   5.561881
11     0.067774  0.409476       5        1   5.436414
5     -0.043857  0.075326       6        1   5.226304
1      0.156577 -0.049675       7        1   4.625300
13     0.028651 -0.087852       8        1   4.129477
6      0.063634  0.122692       9        1   4.042837
17     0.162566 -0.121777      10        1   3.920624
7      0.025112 -0.042701      11        1   3.662566
23    -0.234256  0.016961      12        1   3.373694
3     -0.086738 -0.071588      13        1   3.351326
19     0.108777  0.066393      14        1   3.223110
21     0.011612 -0.109703      15        1   3.192921
9     -0.167013 -0.016998      16        1   2.821492
2      0.117502 -0.089271      17        1   2.734707
15    -0.164444 -0.041030      18        1   2.670536
16     0.091664 -0.016215      19        1   2.643193
8      0.104015 -0.069962      20        1   2.445148
4      0.030945 -0.064053      21        1   2.411266
20     0.110426 -0.023974      22        1   2.066625
22     0.108763  0.041192      23        1   2.063420
10    -0.186244 -0.031414      24        1   2.051750
24     0.041073 -0.078865      25        1   1.962364, topic_info=           Term         Freq        Total Category  logprob  loglift
106      direct  1139.000000  1139.000000  Default  30.0000  30.0000
108     message  1215.000000  1215.000000  Default  29.0000  29.0000
35     register   856.000000   856.000000  Default  28.0000  28.0000
29          hey  2103.000000  2103.000000  Default  27.0000  27.0000
14         help  1391.000000  1391.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
118  experience    18.799115   235.049594  Topic25  -4.4773   1.4050
7       support    19.485738   447.264865  Topic25  -4.4414   0.7976
93         work    18.132117   271.191100  Topic25  -4.5134   1.2259
30       kindly    17.292848  1292.974195  Topic25  -4.5608  -0.3834
3         india    15.916314  1616.809844  Topic25  -4.6438  -0.6898

[1525 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
212      23  0.954219         +
969      18  0.821561        .i
2731     13  0.938153         0
70        1  0.030463         1
70        3  0.010154         1
...     ...       ...       ...
3671      2  0.888220  वनइंडिया
2828     12  0.854626        से
3672      2  0.888219     हिंदी
2794     12  0.952720        है
2389     16  0.413743         −

[2884 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 13, 1, 15, 12, 6, 2, 14, 7, 18, 8, 24, 4, 20, 22, 10, 3, 16, 17, 9, 5, 21, 23, 11, 25])

#####1.5 Topics = 30

In [81]:
#We have had a reduction from .39 to .35 in coherence score 
#by going from 20 to 25. Let's try 30 topics and see what 
#coherence score we'll get. 

# Timing Start
model_1_5_start_time = time.time()

model_1_5 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=30,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_5_end_time = time.time()


In [82]:
#Printing First Model Time
model_1_5_runtime = round(model_1_5_end_time - model_1_5_start_time, 2)
print(model_1_5_runtime)


48.45


In [83]:
# Filtering for words 
words_1_5 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_5.print_topics()]

# Create Topics
topics_1_5 = [' '.join(t[0:10]) for t in words_1_5]

# Getting the topics
for id, t in enumerate(topics_1_5): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
uber service read available safety guideline city government resume mean

------ Topic 1 ------
issue provide hey look payment able detail time dm face

------ Topic 2 ------
action understand driver drive bring uber appropriate instance monitor basis

------ Topic 3 ------
uber sure good able close time bhaiya business live rental

------ Topic 4 ------
india uber ac service thank ride ok pune driver turn

------ Topic 5 ------
thank sir day team great okay wow month uber receive

------ Topic 6 ------
india uber thanks ride sir support come ola cancellation charge

------ Topic 7 ------
drop location driver uber safety use right guy away app

------ Topic 8 ------
book pay ask india day uber reach destination take need

------ Topic 9 ------
trip time date kindly reference kind experience detail share request

------ Topic 10 ------
travel hey kindly appreciate essential allow uber support cab mumbai

------ Topic 11 ------
new health make simple manage seamless

In [84]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_5_perplexity = model_1_5.log_perplexity(corpus)
print('\nPerplexity: ', model_1_5_perplexity) 

# Compute Coherence Score
coherence_model_1_5 = CoherenceModel(model=model_1_5, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_5 = coherence_model_1_5.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_5)



Perplexity:  -6.311927894526822

Coherence Score:  0.47283554071016026


In [85]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_5, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.329153 -0.038740       1        1  11.724497
0     -0.126097 -0.225484       2        1   6.664746
10     0.299645  0.007913       3        1   5.991574
11    -0.080742 -0.005251       4        1   5.375501
1     -0.192330  0.062468       5        1   5.072241
5     -0.004007 -0.114140       6        1   4.532823
14    -0.182533 -0.006361       7        1   4.304152
17    -0.200407  0.035612       8        1   3.843948
13     0.038451 -0.014588       9        1   3.796691
23     0.310179 -0.016418      10        1   3.636779
6     -0.064443 -0.275991      11        1   3.612557
29    -0.017554 -0.221906      12        1   2.760943
3      0.181576  0.031864      13        1   2.749510
28    -0.023236  0.092281      14        1   2.665474
26    -0.009363 -0.092460      15        1   2.602986
22     0.048690 -0.038316      16        1   2.404912
21    -0.109086  0.091142      17        1   2.336405
19    -0.148484  0.034572      18        1   2.284083
2     -0.158096  0.078418      19        1   2.278628
25    -0.048844  0.101692      20        1   2.272522
9     -0.070356  0.076065      21        1   2.266256
27     0.107579  0.089778      22        1   2.262857
4     -0.120726  0.041783      23        1   2.244645
15     0.163433  0.083088      24        1   2.167893
8     -0.095425  0.107680      25        1   2.025425
16    -0.121718  0.012299      26        1   1.980033
7      0.063786  0.091036      27        1   1.965669
24     0.182865  0.013740      28        1   1.830528
20    -0.009095 -0.053417      29        1   1.383078
12     0.057185  0.051641      30        1   0.962646, topic_info=            Term         Freq        Total Category  logprob  loglift
106       direct  1150.000000  1150.000000  Default  30.0000  30.0000
35      register   875.000000   875.000000  Default  29.0000  29.0000
108      message  1227.000000  1227.000000  Default  28.0000  28.0000
29           hey  2226.000000  2226.000000  Default  27.0000  27.0000
30        kindly  1300.000000  1300.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1361        modi     9.731897    22.895228  Topic30  -4.4235   3.7877
25          uber    11.101006  2163.319200  Topic30  -4.2919  -0.6291
3          india    10.499449  1577.936622  Topic30  -4.3476  -0.3693
145   appreciate     9.733822   348.621977  Topic30  -4.4233   1.0649
52          free     9.509720    87.045843  Topic30  -4.4466   2.4291

[1756 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
212      17  0.948493         +
969       8  0.878792        .i
2731     17  0.911407         0
70        2  0.022616         1
70        3  0.067848         1
...     ...       ...       ...
3670     30  0.903076      लेकर
3671     30  0.903097  वनइंडिया
2828     23  0.874296        से
3672     30  0.903098     हिंदी
2794     23  0.931874        है

[3086 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 1, 11, 12, 2, 6, 15, 18, 14, 24, 7, 30, 4, 29, 27, 23, 22, 20, 3, 26, 10, 28, 5, 16, 9, 17, 8, 25, 21, 13])

#####1.6 Topics = 35

In [86]:
# Coherence score jumped back to 0.39, let's try 35 topics
# Timing Start
model_1_6_start_time = time.time()

model_1_6 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=35,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_6_end_time = time.time()


In [87]:
#Printing First Model Time
model_1_6_runtime = round(model_1_6_end_time - model_1_6_start_time, 2)
print(model_1_6_runtime)


50.09


In [88]:
# Filtering for words 
words_1_6 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_6.print_topics()]

# Create Topics
topics_1_6 = [' '.join(t[0:10]) for t in words_1_6]

# Getting the topics
for id, t in enumerate(topics_1_6): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
driver uber ask book wow bangalore option cancel india pay

------ Topic 1 ------
drop location app request outstanding able account balance ride trip

------ Topic 2 ------
hey guideline uber vehicle people priority 2 safety follow distance

------ Topic 3 ------
safe hey phone india travel provide support confirm kindly number

------ Topic 4 ------
hey guideline update mha late car uber accordance service resume

------ Topic 5 ------
team hey trip book need touch issue help step email

------ Topic 6 ------
new health make simple manage device seamless premium style express

------ Topic 7 ------
thank yes india reply sir partner uber team help time

------ Topic 8 ------
driver time uber cancel pune minute place cancellation 20 है

------ Topic 9 ------
uber india mobile help cause time trip lose normal hour

------ Topic 10 ------
help happy guy nice true love day proud fight jai

------ Topic 11 ------
register share message direct hey kindly follow detail 

In [89]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_6_perplexity = model_1_6.log_perplexity(corpus)
print('\nPerplexity: ', model_1_6_perplexity) 

# Compute Coherence Score
coherence_model_1_6 = CoherenceModel(model=model_1_6, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_6 = coherence_model_1_6.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_6)



Perplexity:  -6.3603054794619815

Coherence Score:  0.4156631452524796


In [90]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_6, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
23    -0.301990  0.152606       1        1  9.704607
29    -0.345394 -0.089927       2        1  7.513548
22     0.097004  0.145841       3        1  5.436911
30     0.014109 -0.125550       4        1  5.190784
5      0.103024  0.134417       5        1  4.406682
1      0.145324  0.077150       6        1  4.317621
18    -0.177573  0.077456       7        1  3.886742
14     0.077420  0.078402       8        1  3.731255
6     -0.005756  0.122931       9        1  3.536437
17     0.161568  0.089608      10        1  3.371778
13     0.011512 -0.019037      11        1  2.965193
27    -0.042848  0.084993      12        1  2.808116
19     0.149222  0.024097      13        1  2.807125
31    -0.277320  0.076585      14        1  2.767175
0     -0.011550  0.114208      15        1  2.712560
15     0.140371  0.057147      16        1  2.389838
2     -0.128677  0.142518      17        1  2.372923
24    -0.241754 -0.039560      18        1  2.348955
12    -0.190174 -0.134425      19        1  2.223922
11     0.168314  0.001360      20        1  2.116184
9      0.028927 -0.005707      21        1  2.021452
4      0.138063  0.016852      22        1  1.981206
32     0.102993 -0.137053      23        1  1.940984
8      0.072447 -0.166160      24        1  1.824493
25    -0.010807 -0.072817      25        1  1.821887
3      0.074671  0.005298      26        1  1.661760
16     0.006402  0.072997      27        1  1.618999
21    -0.063237 -0.034893      28        1  1.546888
20     0.094852 -0.015267      29        1  1.428640
10    -0.015095 -0.102325      30        1  1.417671
7      0.099249 -0.007093      31        1  1.366045
26     0.031492 -0.043518      32        1  1.354071
28     0.000689 -0.162733      33        1  1.258192
33    -0.040164 -0.196718      34        1  1.247169
34     0.134683 -0.121682      35        1  0.902187, topic_info=         Term         Freq        Total Category  logprob  loglift
14       help  1524.000000  1524.000000  Default  30.0000  30.0000
106    direct  1081.000000  1081.000000  Default  29.0000  29.0000
108   message  1155.000000  1155.000000  Default  28.0000  28.0000
35   register   768.000000   768.000000  Default  27.0000  27.0000
127   section   677.000000   677.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
25       uber    19.886494  2061.372176  Topic35  -3.6440   0.0670
634     rohit    10.034677    75.586855  Topic35  -4.3280   2.6889
352      hope     9.994607   185.585116  Topic35  -4.3320   1.7866
148     thank    10.355716   470.011488  Topic35  -4.2965   0.8929
4        know    10.127094   284.255283  Topic35  -4.3188   1.3734

[2015 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
212       3  0.861171             +
212      22  0.053823             +
969      16  0.887789            .i
656       6  0.718828  .twitter.com
656      13  0.263570  .twitter.com
...     ...       ...           ...
2828     22  0.889633            से
3672      1  0.916909         हिंदी
2794     22  0.949678            है
3633     29  0.418161         ₹2944
2389     21  0.801832             −

[3452 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[24, 30, 23, 31, 6, 2, 19, 15, 7, 18, 14, 28, 20, 32, 1, 16, 3, 25, 13, 12, 10, 5, 33, 9, 26, 4, 17, 22, 21, 11, 8, 27, 29, 34, 35])

#####1.7 Topics = 40

In [91]:
# Let's try 40 topics
# Timing Start
model_1_7_start_time = time.time()

model_1_7 = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=40,
                       random_state=42,
                       chunksize=2000,
                       passes=10)

# Timing End
model_1_7_end_time = time.time()


In [92]:
#Printing First Model Time
model_1_7_runtime = round(model_1_7_end_time - model_1_7_start_time, 2)
print(model_1_7_runtime)


45.19


In [93]:
# Filtering for words 
words_1_7 = [re.findall(r'"([^"]*)"',t[1]) for t in model_1_7.print_topics()]

# Create Topics
topics_1_7 = [' '.join(t[0:10]) for t in words_1_7]

# Getting the topics
for id, t in enumerate(topics_1_7): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")


------ Topic 0 ------
great drive thank news india uber ola legal है world

------ Topic 1 ------
trip kind experience detail hey direct message registered date time

------ Topic 2 ------
uber shall okay help people nation india say thank driver

------ Topic 3 ------
help trip happy fare hey app refund section kindly refer

------ Topic 4 ------
thank india uber help ur support good great lose hour

------ Topic 5 ------
help way concern app send section hey response kindly address

------ Topic 6 ------
driver safety well rider ensure ask possible switch mode saferforeachother

------ Topic 7 ------
number time driver ride uber d place day minute abhishek

------ Topic 8 ------
good job great price guy wow sell post booking pakistan

------ Topic 9 ------
early update number hey kindly message share register direct follow

------ Topic 10 ------
message direct register hey share detail kindly follow sorry hear

------ Topic 11 ------
issue uber safety hey provide detail know change 

In [94]:
# Compute Perplexity
# a measure of how good the model is. lower the better
model_1_7_perplexity = model_1_7.log_perplexity(corpus)
print('\nPerplexity: ', model_1_7_perplexity) 

# Compute Coherence Score
coherence_model_1_7 = CoherenceModel(model=model_1_7, texts=df['lemma_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_1_7 = coherence_model_1_7.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_1_7)



Perplexity:  -6.333637765414876

Coherence Score:  0.44280368554788263


In [95]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model_1_7, corpus, id2word)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.337471  0.023225       1        1  11.718952
34     0.064047 -0.300939       2        1   5.337378
11     0.040281 -0.064381       3        1   5.072491
14     0.127479 -0.087163       4        1   4.653998
36     0.017244 -0.241563       5        1   3.995844
19     0.138722 -0.124312       6        1   3.541795
24    -0.311027 -0.017648       7        1   3.463649
12    -0.289367 -0.041925       8        1   3.301836
29    -0.194182  0.024869       9        1   2.604469
13    -0.069929  0.063791      10        1   2.586076
37    -0.226890  0.055062      11        1   2.579667
25     0.052767  0.082370      12        1   2.576282
10     0.038074  0.076062      13        1   2.568049
1      0.142494 -0.024083      14        1   2.475768
5      0.010925 -0.019544      15        1   2.404351
39     0.166076  0.083601      16        1   2.388254
2      0.098946 -0.162460      17        1   2.354332
35     0.101781  0.040504      18        1   2.114452
3     -0.126105 -0.023968      19        1   2.103811
17     0.146133  0.104247      20        1   2.092057
0     -0.009291 -0.170886      21        1   2.059144
31    -0.282054  0.014243      22        1   2.033716
7     -0.028982 -0.010648      23        1   1.959682
4      0.054269  0.114951      24        1   1.928345
27     0.107195  0.130352      25        1   1.845575
8      0.157911  0.067764      26        1   1.805716
21     0.108708  0.077610      27        1   1.768708
22    -0.069574  0.043046      28        1   1.745258
15     0.055373 -0.029956      29        1   1.615702
26    -0.044638  0.017663      30        1   1.613973
33     0.060630 -0.006657      31        1   1.383711
9      0.003886 -0.072906      32        1   1.252962
28     0.050772  0.069126      33        1   1.233961
38    -0.054378  0.095792      34        1   1.193187
20     0.081798  0.143884      35        1   1.173967
16     0.114034 -0.049802      36        1   1.168393
30     0.144498  0.032983      37        1   1.123532
32     0.024875  0.047393      38        1   1.093063
23    -0.191245  0.012341      39        1   1.058294
6      0.126219  0.027963      40        1   1.009602, topic_info=          Term         Freq        Total Category  logprob  loglift
106     direct  1154.000000  1154.000000  Default  30.0000  30.0000
35    register   879.000000   879.000000  Default  29.0000  29.0000
108    message  1224.000000  1224.000000  Default  28.0000  28.0000
29         hey  2234.000000  2234.000000  Default  27.0000  27.0000
14        help  1423.000000  1423.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
175     driver    16.406749  1092.671970  Topic40  -3.9488   0.3969
1162   believe     9.375224    35.024396  Topic40  -4.5085   3.2776
570       stay     9.236567    67.342037  Topic40  -4.5234   2.6090
154   customer     9.232025   130.950652  Topic40  -4.5239   1.9435
45      safety     8.742304   432.355096  Topic40  -4.5784   0.6945

[2236 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
212      40  0.941744             +
969      17  0.865508            .i
656       6  0.262243  .twitter.com
656      14  0.476805  .twitter.com
656      16  0.214562  .twitter.com
...     ...       ...           ...
2828     37  0.864329            से
3672     39  0.907811         हिंदी
2794     37  0.967177            है
3633     33  0.630081         ₹2944
2389     27  0.667095             −

[3795 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 35, 12, 15, 37, 20, 25, 13, 30, 14, 38, 26, 11, 2, 6, 40, 3, 36, 4, 18, 1, 32, 8, 5, 28, 9, 22, 23, 16, 27, 34, 10, 29, 39, 21, 17, 31, 33, 24, 7])

#####1.8 Topics = 5-200


In [96]:
#Defining a function to loop over number of topics to be used to find an 
#optimal number of tipics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the 
    LDA model with respective number of topics
    """
    coherence_values_topic = []
    model_list_topic = []
    for num_topics in range(start, limit, step):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list_topic.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values_topic.append(coherencemodel.get_coherence())

    return model_list_topic, coherence_values_topic      

In [97]:
# Can take a long time to run.
model_1_8_start_time = time.time()

model_list_topic, coherence_values_topic = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=df['lemma_tokens'],
                                                        start=2, limit=200, step=6)
model_1_8_end_time = time.time()


In [98]:
#Printing First Model Time
model_1_8_runtime = round(model_1_8_end_time - model_1_8_start_time, 2)
print(model_1_8_runtime)


1414.43


In [99]:
limit=200; start=2; step=6;
x_topic = range(start, limit, step)

topic_ts = {'coherence_value': coherence_values_topic,
            'number_of_topics': x_topic}

topic_chart = pd.DataFrame(data=topic_ts)

topic_fig = px.line(topic_chart, x="number_of_topics", y="coherence_value")
topic_fig.show()
